In [28]:
import os

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyarrow.feather as feather
import seaborn as sns
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller, coint
from scipy.optimize import minimize
from datetime import datetime, timedelta
from IPython.core.interactiveshell import InteractiveShell
from matplotlib.ticker import ScalarFormatter

InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [66]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
mpl.rcParams['font.size'] = 16  # 기본 폰트 크기 설정
mpl.rcParams['axes.titlesize'] = 20  # 제목 폰트 크기 설정
mpl.rcParams['axes.labelsize'] = 18  # 축 레이블 폰트 크기 설정
mpl.rcParams['xtick.labelsize'] = 14  # x축 눈금 폰트 크기 설정
mpl.rcParams['ytick.labelsize'] = 14  # y축 눈금 폰트 크기 설정
mpl.rcParams['legend.fontsize'] = 16  # 범례 폰트 크기 설정

In [112]:
# price_df = feather.read_feather('data\\91_coin_price.feather')
# volume_df = feather.read_feather('data\\91_coin_volume.feather')
price_df = feather.read_feather('total_price_data.feather')
volume_df = feather.read_feather('total_volume_data.feather')

In [113]:
n_cols = [i[4:] for i in price_df.columns]
price_df.columns = n_cols
volume_df.columns = n_cols
agg_dict = {col:'last' for col in price_df.columns}

In [114]:
price_df

,BTC,1INCH,MOC,CBK,MASK,IMX,ELF,IQ,FLOW,HUNT,...,ARDR,QKC,QTUM,STMX,NEO,XLM,MVL,CVC,SEI,STRAX
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,19010000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,73000.0000,NaN,102750.0000,540.0000,NaN,NaN,NaN,1884.0000
2018-01-01 01:00:00,18999000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72440.0000,NaN,103150.0000,531.0000,NaN,NaN,NaN,1895.0000
2018-01-01 02:00:00,19015000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72600.0000,NaN,105650.0000,571.0000,NaN,NaN,NaN,1900.0000
2018-01-01 03:00:00,19111000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,73480.0000,NaN,105450.0000,586.0000,NaN,NaN,NaN,1960.0000
2018-01-01 04:00:00,19244000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72980.0000,NaN,104950.0000,620.0000,NaN,NaN,NaN,1971.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-15 19:00:00,93735000.0000,590.5000,104.4000,893.6000,4024.0000,2476.0000,586.2000,10.7800,1022.0000,480.7000,...,111.0000,13.6500,4167.0000,8.5350,17850.0000,139.0000,6.5030,177.7000,592.9000,83.9600
2024-06-15 20:00:00,93678000.0000,588.6000,104.0000,896.4000,4020.0000,2464.0000,586.9000,10.7200,1013.0000,478.9000,...,110.5000,13.7100,4169.0000,8.4900,17810.0000,138.7000,6.5040,178.0000,588.7000,83.7300
2024-06-15 21:00:00,93749000.0000,588.7000,104.2000,896.3000,4017.0000,2471.0000,584.8000,10.7400,1014.0000,480.9000,...,110.2000,13.6800,4168.0000,8.5160,17880.0000,138.6000,6.5010,177.5000,590.7000,83.7400


In [115]:
price_df = price_df.resample('1D').agg(agg_dict)

In [116]:
price_df

,BTC,1INCH,MOC,CBK,MASK,IMX,ELF,IQ,FLOW,HUNT,...,ARDR,QKC,QTUM,STMX,NEO,XLM,MVL,CVC,SEI,STRAX
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01,18860000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72610.0000,NaN,109900.0000,686.0000,NaN,NaN,NaN,2211.0000
2018-01-02,20263000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,73320.0000,NaN,124000.0000,762.0000,NaN,NaN,NaN,2197.0000
2018-01-03,20900000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72190.0000,NaN,143900.0000,1220.0000,NaN,NaN,NaN,2299.0000
2018-01-04,23402000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,70700.0000,NaN,153550.0000,1130.0000,NaN,NaN,NaN,2450.0000
2018-01-05,27444000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,81600.0000,NaN,153800.0000,1075.0000,NaN,NaN,NaN,2420.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-11,95329000.0000,576.5000,107.8000,915.5000,4265.0000,2593.0000,595.6000,11.2600,1061.0000,503.3000,...,114.7000,13.3600,4208.0000,8.8050,18320.0000,137.4000,6.7520,175.2000,647.9000,85.9900
2024-06-12,95934000.0000,597.2000,109.3000,919.4000,4372.0000,2702.0000,606.6000,11.6200,1096.0000,506.4000,...,114.9000,13.7000,4306.0000,9.1200,19070.0000,140.5000,6.9190,184.0000,674.2000,87.5800
2024-06-13,94614000.0000,585.9000,107.8000,899.0000,4183.0000,2546.0000,654.9000,11.0500,1037.0000,489.4000,...,111.3000,13.5600,4211.0000,8.7700,18480.0000,138.2000,6.7300,176.5000,627.8000,83.8300


In [128]:
def get_formation_trading_df(price_df, formation_start_datetime, gap=False):
    formation_start = formation_start_datetime
    formation_end = formation_start + pd.DateOffset(months=12) - pd.Timedelta(days=1)
    trading_start = (formation_end + pd.Timedelta(days=1) if gap == False 
                                     else formation_end + pd.Timedelta(days=2))
    trading_end = trading_start + pd.DateOffset(months=6) - pd.Timedelta(days=1)
    log_price_df = np.log(price_df)
    tdf = log_price_df[formation_start:trading_end]
    tdf = tdf.dropna(axis=1)
    formation_df = tdf[formation_start:formation_end]
    trading_df = tdf[trading_start:trading_end]
    return formation_df, trading_df

In [129]:
gap = False
cnt = 0

formation_start = price_df.index[0]
last_datetime = price_df.index[-1]
trading_end = formation_start + pd.Timedelta(days=540)
# print('\t\tFormation Period\t\t\t      Trading Period')

while 1:
    cnt += 1
    formation_end = formation_start +pd.DateOffset(months=12) - pd.Timedelta(days=1)
    trading_start = (formation_end + pd.Timedelta(days=1) if gap == False 
                                     else formation_end + pd.Timedelta(days=2))
    trading_end = trading_start + pd.DateOffset(months=6) - pd.Timedelta(days=1)
    
    if trading_end > last_datetime:
        break
    
    formation_df, trading_df = get_formation_trading_df(price_df, formation_start)
    print(f'#{cnt}:\t {formation_start} ~ {formation_end}\t{trading_start} ~ {trading_end}')
    
    corr_df = formation_df.corr()
    corr_df = corr_df.reset_index().melt(id_vars='index', value_vars=corr_df.columns)
    corr_df.columns =['coin1', 'coin2', 'corr']
    corr_df = corr_df[corr_df['corr']!=1].sort_values(by='corr', ascending=False).drop_duplicates(subset='corr')
    corr_df = corr_df.reset_index(drop=True)

    adf_df = {
        "ADF Test Statistic": [],
        'p-value': [],
        'Half Life': []
    }
    for pair in corr_df.values:
        coin1, coin2, _ = pair
        coin1_df = formation_df[coin1]
        coin2_df = formation_df[coin2]
        coin1_log_price = coin1_df.values
        coin2_log_price = coin2_df.values
        
        # 선형 회귀 모델 적합
        X = coin1_log_price
        Y = coin2_log_price

        model = sm.OLS(Y, X).fit()
        beta = model.params
        ut = Y - (beta * X)
        
        adf_res = adfuller(ut)
        model = sm.tsa.ARIMA(ut, order=(1,0,0)).fit()
        phi_hat = model.arparams[0]
        half_life = np.log(2) / np.log(1 / abs(phi_hat))
        
        adf_df["ADF Test Statistic"].append(adf_res[0])
        adf_df['p-value'].append(adf_res[1])
        adf_df['Half Life'].append(half_life)
        
    adf_df = pd.DataFrame(adf_df)
    corr_df = pd.concat([corr_df, adf_df], axis=1)
    corr_df = corr_df.loc[corr_df['p-value'] < 0.05, :].reset_index(drop=True)
    corr_df['corr rank'] = corr_df['corr'].rank(ascending=False)
    corr_df['p-value rank'] = corr_df['p-value'].rank()
    corr_df['Half Life rank'] = corr_df['Half Life'].rank()
    corr_df['total rank'] = corr_df['corr rank'] + corr_df['p-value rank'] + corr_df['Half Life rank']
    corr_df = corr_df.sort_values(by='total rank').reset_index(drop=True)
    
    print(f'최종 {corr_df.shape[0]}개 페어 남음')

    corr_df.to_csv(f'data2/corr_{cnt}.csv', index=False)
    
    del corr_df
    
    formation_start += pd.DateOffset(months=1)

#1:	 2018-01-01 00:00:00 ~ 2018-12-31 00:00:00	2019-01-01 00:00:00 ~ 2019-06-30 00:00:00
최종 33개 페어 남음
#2:	 2018-02-01 00:00:00 ~ 2019-01-31 00:00:00	2019-02-01 00:00:00 ~ 2019-07-31 00:00:00
최종 23개 페어 남음
#3:	 2018-03-01 00:00:00 ~ 2019-02-28 00:00:00	2019-03-01 00:00:00 ~ 2019-08-31 00:00:00
최종 14개 페어 남음
#4:	 2018-04-01 00:00:00 ~ 2019-03-31 00:00:00	2019-04-01 00:00:00 ~ 2019-09-30 00:00:00
최종 23개 페어 남음
#5:	 2018-05-01 00:00:00 ~ 2019-04-30 00:00:00	2019-05-01 00:00:00 ~ 2019-10-31 00:00:00
최종 34개 페어 남음
#6:	 2018-06-01 00:00:00 ~ 2019-05-31 00:00:00	2019-06-01 00:00:00 ~ 2019-11-30 00:00:00
최종 65개 페어 남음
#7:	 2018-07-01 00:00:00 ~ 2019-06-30 00:00:00	2019-07-01 00:00:00 ~ 2019-12-31 00:00:00
최종 48개 페어 남음
#8:	 2018-08-01 00:00:00 ~ 2019-07-31 00:00:00	2019-08-01 00:00:00 ~ 2020-01-31 00:00:00
최종 43개 페어 남음
#9:	 2018-09-01 00:00:00 ~ 2019-08-31 00:00:00	2019-09-01 00:00:00 ~ 2020-02-29 00:00:00
최종 27개 페어 남음
#10:	 2018-10-01 00:00:00 ~ 2019-09-30 00:00:00	2019-10-01 00:00:00 ~ 2020-03-31 0

In [ ]:
top_n = 5

rolling_num = 0

total_return = np.zeros(60) # rolling 평균 성과
total_std = np.zeros(60)

cnt = 0

formation_start = price_df.index[0]
last_datetime = price_df.index[-1]
trading_end = formation_start + pd.Timedelta(days=540)

while 1:
    cnt += 1
    formation_end = formation_start +pd.DateOffset(months=12) - pd.Timedelta(days=1)
    trading_start = (formation_end + pd.Timedelta(days=1) if gap == False 
                                     else formation_end + pd.Timedelta(days=2))
    trading_end = trading_start + pd.DateOffset(months=6) - pd.Timedelta(days=1)
    
    if trading_end > last_datetime:
        break
    
    formation_df, trading_df = get_formation_trading_df(price_df, formation_start)
    print(f'#{cnt}:\t {formation_start} ~ {formation_end}\t{trading_start} ~ {trading_end}')
    
    top_df = pd.read_csv(f"data2/corr_{i}.csv")
    pair_names = top_df.loc[:top_n-1:][['coin1', 'coin2']]
    pair_names = pair_names.values

    # rolling_total_open = 0
    
    pair_index = 0
    for c1, c2 in pair_names:
        pair_ret = 0
        
        first_asset_name = c1
        second_asset_name = c2
        in_X = formation_df[first_asset_name]
        in_Y = formation_df[second_asset_name]
        out_X = trading_df[first_asset_name]
        out_Y = trading_df[second_asset_name]

        in_model = sm.OLS(in_X, in_Y).fit()
        beta = in_model.params[0]

        in_coint_price = in_X - (beta * in_Y)
        out_coint_price = out_X - (beta * out_Y)

        insample_sd = np.std(in_coint_price)
        index = np.where(
            np.abs(out_coint_price) > (2*insample_sd)
        )[0]

        if len(index) != 0:
            # pair_open += 1
            
            pair_ind = out_coint_price[index[0]] < 0
            
            ind1 = int(pair_ind) - int(not pair_ind)
            ind2 = int(not pair_ind) - int(pair_ind)
            
            W = np.array([
                ind1 * (1 / out_X[index[0]]),
                ind2 * (1 / out_Y[index[0]])
            ])
            
            pair_port = np.dot(W, np.vstack([
                out_X[index[0]:],
                out_Y[index[0]:]
            ]))
            ind_sell = np.where(
                out_coint_price[index[0]:] * out_coint_price[index[0]] <= 0
            )[0]

            if len(ind_sell) == 0:
                pair_ret = pair_port[-1]
            else:
                pair_ret = pair_port[min(ind_sell[0] + 1, len(pair_port) - 1)]
                # pair_n_sell += 1
        else:
            pair_ret = 0

        rolling_pair_return[pair_index] = pair_ret * (1- 0.001) # 수수료 계산
        pair_index += 1
        
    total_return[cnt-1] = np.mean(rolling_pair_return)
    total_std[cnt-1] = np.std(rolling_pair_return)


In [ ]:
np.mean(total_return) / 42 * 365
np.mean(total_std) / np.sqrt(42 * 365)

In [ ]:
ind_sell = np.where(out_coint_price[index[0]:] * out_coint_price[index[0]] <= 0)[0]
ind_sell

In [ ]:
min(ind_sell[0]+1, len(pair_port)-1)
pair_port[63]

In [ ]:
ind_sell = 


In [ ]:
plt.plot(pair_port)
plt.title(f'Pair Returns (Window {rolling_num})')
plt.axhline(y=0, color='blue', linestyle='--')
plt.grid(True)

In [ ]:
(-1/out_X[15]) * (out_X[15])
